# Watermark detection – step‑by‑step guide

This notebook helps you check whether images contain a watermark and how robust that watermark is to simple edits. It is written for non‑technical users and works both on your computer and in cloud notebooks.

What you will do:
- Choose a watermarking method (Stable Signature available today; more coming).
- Point the notebook at a folder of images.
- Optionally, provide a folder of the original (non‑watermarked) images for quality comparisons.
- Run detection/decoding and save results as CSV files you can share.


## Before you start

Please make sure you have:
- A folder of images to analyse (for example, `data/watermarked`).
- If you have the originals without watermark, a second folder with the matching images (optional).
- For Stable Signature, a small model file (we will download it for you if missing).

Tip: You do not need to install anything manually if you use this notebook in a managed environment (e.g. cloud). If a library is missing, the next cell will install it for you.

In [ ]:
# This cell quietly ensures required libraries are available.
import importlib.util, sys, subprocess

def ensure(pkg: str, pip_name: str = None):
    if importlib.util.find_spec(pkg) is None:
        print(f"Installing {pip_name or pkg}...")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', pip_name or pkg])

# Core utilities
ensure('pandas')
ensure('torch')
ensure('torchvision')
ensure('skimage', 'scikit-image')
ensure('pytorch_fid')

print('Libraries are ready.')


## Choose your settings

Please edit only the values on the right‑hand side of each line. If you are unsure, leave the defaults as they are. All paths can be absolute (beginning with `/`) or relative to this notebook file.

Notes:
- `model_name`: "stable_signature" is supported today. New models can be added later without changing this notebook.
- `img_dir_nw`: optional folder containing the original, non‑watermarked images (for quality comparison).
- `key_str`: for Stable Signature, supply the bitstring if you want accuracy against the key. Leave blank to only decode bits (set `decode_only` to True).
- `attack_mode`: "none" (no edits), "few" (a small, realistic set), or "all" (thorough).


In [ ]:
# Settings – change these as needed
model_name = 'stable_signature'           # Options: 'stable_signature' (more coming soon)
img_dir = 'data/watermarked'             # Folder of images to analyse
img_dir_nw = None                        # Optional folder of original, non‑watermarked images
output_dir = 'output/notebook'           # Where to save the CSV results

# Stable Signature only
key_str = '111010110101000001010111010011010100010000100111'  # Leave '' if unknown and set decode_only=True
decode_only = False                        # True: only decode bits; False: compute accuracy against key
attack_mode = 'few'                        # 'none' | 'few' | 'all'
msg_decoder_path = 'models/dec_48b_whit.torchscript.pt'  # Will be downloaded below if missing


## Prepare model files (Stable Signature)

If you selected Stable Signature and the small decoder file is missing, this cell will download it automatically.

In [ ]:
import os, urllib.request
from pathlib import Path

if model_name == 'stable_signature':
    os.makedirs(os.path.dirname(msg_decoder_path), exist_ok=True)
    if not Path(msg_decoder_path).exists():
        url = 'https://dl.fbaipublicfiles.com/ssl_watermarking/dec_48b_whit.torchscript.pt'
        print('Downloading Stable Signature decoder (once only)...')
        urllib.request.urlretrieve(url, msg_decoder_path)
        print('Saved to', msg_decoder_path)
    else:
        print('Decoder file already present at', msg_decoder_path)
else:
    print('No model files needed for this selection.')


## Run the evaluation

This cell analyses your images and saves two CSV files in `output/notebook` (or your chosen folder):
- `log_stats.csv`: detection/bit results per image (and per edit).
- `img_metrics.csv`: optional image‑to‑image quality metrics (SSIM, PSNR, Linf) if you provided `img_dir_nw`.

You will also see a preview of the first few rows.

In [ ]:
import os
import pandas as pd

# Use the simple pluggable interface added to the repository
from watermarking.registry import get_model
import run_evals as _evals  # for optional quality metrics

os.makedirs(output_dir, exist_ok=True)

model = get_model(model_name)
prepare_kwargs = {}
if model_name == 'stable_signature':
    prepare_kwargs['msg_decoder_path'] = msg_decoder_path
model.prepare(**prepare_kwargs)

rows = model.evaluate_images(
    img_dir=img_dir,
    key_str=None if decode_only else key_str,
    decode_only=decode_only,
    attack_mode=attack_mode,
    batch_size=32,
)
df = pd.DataFrame(rows)
df.to_csv(os.path.join(output_dir, 'log_stats.csv'), index=False)
print('Saved detection/bit results to', os.path.join(output_dir, 'log_stats.csv'))
df.head()


In [ ]:
# Optional: compute image‑to‑image quality metrics if you have non‑watermarked originals
import numpy as np

if img_dir_nw:
    print('Computing image similarity metrics (SSIM / PSNR / Linf)...')
    img_metrics = _evals.get_img_metric(img_dir, img_dir_nw)
    img_df = pd.DataFrame(img_metrics)
    img_df.to_csv(os.path.join(output_dir, 'img_metrics.csv'), index=False)
    print('Saved image metrics to', os.path.join(output_dir, 'img_metrics.csv'))
    # Small summary
    ssims, psnrs, linfs = img_df['ssim'], img_df['psnr'], img_df['linf']
    print(f"SSIM: {np.mean(ssims):.4f} ± {np.std(ssims):.4f}")
    print(f"PSNR: {np.mean(psnrs):.4f} ± {np.std(psnrs):.4f}")
    print(f"Linf: {np.mean(linfs):.4f} ± {np.std(linfs):.4f}")
else:
    print('Skipped image metrics (no non‑watermarked folder provided).')


## What next?

- Share the CSV files from your output folder via email or chat.
- Re‑run with a different `attack_mode` for a more thorough test.
- When additional watermarking models are added to this repository, you will be able to select them by name in the settings cell without further changes.
